In [17]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [18]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    '''pass in list of messages covering different roles'''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    #print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [7]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages

Single chat messages

In [8]:
messages = context + [{'role':'user', 
                       'content':'Please tell me how many sizes of pizza you have.'}]

get_completion_from_messages(messages)

'We have three sizes of pizza: small, medium, and large.'

In [10]:
# example of hallucination for incomplete data
messages = context + [{'role':'user', 
                       'content':'Please tell me how many sizes of bottled water \
                       you have. How many ounces?'}]

get_completion_from_messages(messages)

'We have only one size of bottled water, which is 16.9 ounces.'

In [11]:
messages = context + [{'role':'user', 
                       'content':'Please tell me the price of a medium pepperoni \
                       pizza with added canadian bacon'}]

get_completion_from_messages(messages)

'A medium pepperoni pizza costs $10.00 and adding Canadian bacon would cost an additional $3.50. So, the total cost would be $13.50.'

In [12]:
# another hallucination

messages = context + [{'role':'user',
                       'content':'is it possible to top a cheese \
                        pizza with the greek salad?'}]

get_completion_from_messages(messages)

"I'm sorry, but we don't offer that option. However, we do have a Greek pizza that comes with feta cheese, olives, and tomatoes. Would you like to order that instead?"

Try to correct for hallucinations by adjusting adding the requirement that it not make up details it does not know.

In [15]:
context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
If the customer asks for details you do not know then admit you \
do not know rather than make something up.
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water (size 12 ounces) 5.00 \
"""} ]  # accumulate messages

In [19]:
# try the water question again after adjusting menu size
# it does not make things up this time
messages = context + [{'role':'user', 
                       'content':'Please tell me how many sizes of bottled water \
                       you have. How many ounces?'}]

get_completion_from_messages(messages)

'We have only one size of bottled water which is 12 ounces.'

In [20]:
# but the problem of hallucination still exists for other menu items
messages = context + [{'role':'user', 
                       'content':'Please tell me how many sizes of sprite \
                       you have. How many ounces?'}]

get_completion_from_messages(messages)

'We have three sizes of Sprite: small (1.00), medium (2.00), and large (3.00). The sizes are 12 ounces, 20 ounces, and 2 liters, respectively.'

In [21]:
messages = context + [{'role':'user', 
                       'content':'Please tell me how many sizes of cheese pizza \
                       you have. What is the diameter of each in inches?'}]

get_completion_from_messages(messages)

'We have three sizes of cheese pizza: small (6.50 inches), medium (9.25 inches), and large (10.95 inches).'